# App Subscription Project

by Carolina S. Lopes

**STRV DATA SCIENCE ACADEMY**

## Defining the business problem

## Data Acquisition

#### Importing libraries

We'll start our analysis by loading all necessary Python libraries in order to perform our project:

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

# ---
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

#### Loading dataset

The project's main dataset is available on the Kaggle platform. 
In order to download it, please check this [link](https://www.kaggle.com/datasets/hkhamnakhalid/customers-to-subscription-through-app-behavior).

After downloading the dataset, you need to provide the filepath in order to read it with the help of pandas library.

In [2]:
df = pd.read_csv('/Users/csergilo/Desktop/strv-ds-academy/app-subscription-strv-academy/datasets/FineTech_appData.csv')

Let's check if the loaded dataset is consistent to what is available on Kaggle page.

In [3]:
df.head()

user               first_open  dayofweek       hour  age  \
0  235136  2012-12-27 02:14:51.273          3   02:00:00   23   
1  333588  2012-12-02 01:16:00.905          6   01:00:00   24   
2  254414  2013-03-19 19:19:09.157          1   19:00:00   23   
3  234192  2013-07-05 16:08:46.354          4   16:00:00   28   
4   51549  2013-02-26 18:50:48.661          1   18:00:00   31   

                                         screen_list  numscreens  minigame  \
0  idscreen,joinscreen,Cycle,product_review,ScanP...          15         0   
1  joinscreen,product_review,product_review2,Scan...          13         0   
2                                  Splash,Cycle,Loan           3         0   
3  product_review,Home,product_review,Loan3,Finan...          40         0   
4  idscreen,joinscreen,Cycle,Credit3Container,Sca...          32         0   

   used_premium_feature  enrolled            enrolled_date  liked  
0                     0         0                      NaN      0  
1                     0         0                      NaN      0  
2                     1         0                      NaN      1  
3                     0         1  2013-07-05 16:11:49.513      0  
4                     0         1  2013-02-26 18:56:37.841      1

Great! The dataset was uploaded as expected. We can now proceed with the Data Wrangling workflow.

---

## Data Wrangling

The number of rows and columns of the dataframe can be checked with the `pd.DataFrame.shape` method as below, and the output is given in the format (rows, columns)

In [4]:
df.shape

(50000, 12)

The dataframe columns are listed below:

In [5]:
df.columns

Index(['user', 'first_open', 'dayofweek', 'hour', 'age', 'screen_list',
       'numscreens', 'minigame', 'used_premium_feature', 'enrolled',
       'enrolled_date', 'liked'],
      dtype='object')

One important step during Data Wrangling consists in checking the datatypes for each column, and confirm if they are correct. In case they aren't, we need to cast our data to the correct datatype.

In [6]:
df.dtypes

user                     int64
first_open              object
dayofweek                int64
hour                    object
age                      int64
screen_list             object
numscreens               int64
minigame                 int64
used_premium_feature     int64
enrolled                 int64
enrolled_date           object
liked                    int64
dtype: object

Another important step is to check whether we have any missing values on our columns:

In [7]:
df.isnull().sum()

user                        0
first_open                  0
dayofweek                   0
hour                        0
age                         0
screen_list                 0
numscreens                  0
minigame                    0
used_premium_feature        0
enrolled                    0
enrolled_date           18926
liked                       0
dtype: int64

As given above, we have many missing entries in the "enrolled_date" column. This makes sense, as no date should appear in this field if the user didn't subscribe to the app Premium features.

Missing data can also appear as "0". Depending on our fields, this may affect the output of any statistical calculations for that column.

In the case of our project, a column that could present zero values that don't make sense corresponds to the "age". Or are there any babies dealing with FinTech apps already? 🤔

In [8]:
sum([value == 0 for value in df['age']])

0

Okay, that seems good! Just to be sure, let's also check if any users haven't actually interacted with the app (that is, if there's any entry where `df['numscreens']` is equal to zero).

In [9]:
sum([value == 0 for value in df['numscreens']])

0

As nothing seems suspicious with our dataset, let's check our unique values! With this step, we can infer if our columns correspond to numerical or categorical variables.

In [10]:
df.nunique()

user                    49874
first_open              49747
dayofweek                   7
hour                       24
age                        78
screen_list             38799
numscreens                151
minigame                    2
used_premium_feature        2
enrolled                    2
enrolled_date           31001
liked                       2
dtype: int64

Interesting! It seems that we have some non-unique users in our dataset. We can check for duplicate user entries in our dataframe:

In [11]:
df.duplicated(subset=['user']).sum()

126

Now, we can compare that with the amount of duplicated rows:

In [12]:
df.duplicated().sum()

15

We can first remove the duplicated rows, and then check the duplicated users in more detail:

In [13]:
df = df.drop_duplicates()

In [14]:
user = df['user']

df_dupl = df[user.isin(user[user.duplicated()])].sort_values(by = ['user'])
df_dupl.head(6)

user               first_open  dayofweek       hour  age  \
20029  2477  2013-04-15 17:45:44.684          0   17:00:00   29   
27838  2477  2013-04-15 17:45:44.684          0   17:00:00   27   
49052  5867  2013-02-11 08:46:43.530          0   08:00:00   27   
49397  5867  2013-02-11 08:46:43.530          0   08:00:00   26   
49509  8767  2013-05-14 23:24:19.949          1   23:00:00   36   
43600  8767  2013-05-14 23:24:19.949          1   23:00:00   34   

                                             screen_list  numscreens  \
20029                                         Loan2,Loan           3   
27838                                         Loan2,Loan           3   
49052  Cycle,Home,Institutions,SelectInstitution,Bank...          18   
49397  Cycle,Home,Institutions,SelectInstitution,Bank...          18   
49509  Splash,idscreen,Cycle,product_review,product_r...          22   
43600  Splash,idscreen,Cycle,product_review,product_r...          22   

       minigame  used_premium_feature  enrolled            enrolled_date  \
20029         0                     1         0                      NaN   
27838         0                     1         0                      NaN   
49052         0                     0         0                      NaN   
49397         0                     0         0                      NaN   
49509         0                     0         1  2013-05-14 23:25:45.033   
43600         0                     0         1  2013-05-14 23:25:45.033   

       liked  
20029      0  
27838      0  
49052      1  
49397      1  
49509      0  
43600      0

We can see that some of the duplicated users have inconsistent ages. How many non-problematic entries do we have, that is, how many of these rows have consistent user ages?

In [16]:
# users = df_dupl['user'].to_numpy()
# print(sum(users[::2] == users[1::2])) # compares both duplicated user entries. all even and odd entries are equal!

ages = df_dupl['age'].to_numpy()
print(sum(ages[::2] == ages[1::2])) # compares both duplicated user entries

13


This means that most of our duplicated entries have are inconsistent in the user ages. We are then deleting all duplicate user entries and updating our dataset:

In [17]:
df = df.drop_duplicates(subset=['user'])

Resetting the indices:

In [18]:
df.reset_index(drop=True, inplace=True)

Now we're good to go! Focusing back on our categorical and numerical variables:

<details>
<summary> <b> Click here to check the summary of our variables <b> </summary>
    

**Categorical variables**

- **user**: unique values corresponding to user ID

- **dayofweek:** values ranging from 0 ("Monday") to 6 ("Sunday"), corresponding to the day of the week

- **hour:** time category from 00:00:00 to 23:00:00 corresponding to the *first_open* variable

- **minigame:** 
    - 0 → user didn't minigame
    - 1 → user played minigame

- **used_premium_feature:** 
    - 0 → user haven't used a Premium feature
    - 1 → Premium feature was used
        
- **liked:** 
    - 0 → user haven't liked/rated the app
    - 1 → user likes the app

**Numerical Variables**

- **age:** user ages

- **numscreens:** number of screens visited by the user

**Dates**

- **first_open:** time at which user created the account \

- **enrolled_date:** day at which user subscribed to Premium

    
    
</details>

Let's create some new binned categories to enrich our analysis:

In [ ]:
# day periods: (0) night (1) morning (2) afternoon (3) evening

df['hour'] = df['hour'].apply(lambda value: int(value.strip()[0:2]))
df['dayperiod'] = df['hour'].apply(lambda value: ((0 if value >= 0 and value <= 5 else 1) if value <= 11 else 2) if value <= 17 else 3)


In [ ]:
df.enrolled_date = pd.to_datetime(df.enrolled_date)
df.first_open = pd.to_datetime(df.first_open)

df['days_to_enroll'] = round((df.enrolled_date - df.first_open) / pd.Timedelta(days=1))

In [ ]:
# age groups: (0) 18-24 (1) 25-34 (2) 35-44 (3) 45-54 (4) 55-64 (5) 65+

df['age_group'] = df['age'].apply(lambda age: ((((0 if age >= 18 and age <= 24 else 1) if age <= 34 else 2) if age <= 44 else 3) if age <= 54 else 4) if age <= 64 else 5)



In [ ]:
df.head(10)

Our dataframe categories are now as follows:

In [ ]:
df.dtypes

Getting back to our dataframe, there's one column that needs special attention: the "screen_list" column. If we scroll up a bit, we can see that this column has more than 38000 unique values. That is because the entries correspond to strings with all screens that a given user has visited.

To start analysing this data, we can first turn the entries into lists with the screen names

In [ ]:
df['screen_list'] = df['screen_list'].apply(lambda x: x.split(','))

And then check with how many and which unique screen names we're dealing with:

In [ ]:
list_items = df['screen_list'].notnull()
all_screens = set()

for i, item in enumerate (list_items):
        
    if item == True:

            entry = df['screen_list'][i]

            for j in entry:

                # adicionar diferentes itens da coluna em um set
                all_screens.add(j)

In [ ]:
total_screens = len(all_screens)
total_screens

In [ ]:
all_screens

From the output, we can see that there are many similar screens. For example, we can check the screens starting with "Credit":

In [ ]:
[i for i in all_screens if i.startswith("Credit")]

We can group these screen names in order to reduce our data. Let's define a function that will do this for us:

In [ ]:
def group_screens(group):

    for i, item in enumerate(df['screen_list']):

        for j, jtem in enumerate((pd.Series(df['screen_list'][i])).str.startswith(group)):

            if jtem == True:

                df['screen_list'][i][j] = group

The function `group_screens` is able to replace all entries starting with a given string by this string. For example: if we call the function `group_screens('Credit')`, it will replace all string starting with "Credit" by "Credit".

In [ ]:
group_screens('Credit')

We can do the same for other categories:

In [ ]:
screens_to_group = ['Loan', 'Profile', 'Saving', 'Verify', 'product_review']

for s in screens_to_group:
    
    group_screens(s)

Our dataset is now as follows:

In [ ]:
df.head()

To enrich our analysis, we can add some numerical columns to our dataset corresponding to screen counts of the screen_list:

In [ ]:
df['total_screens'] = df['screen_list'].apply(lambda value: len(value))

In [ ]:
def count_screens(name):
    
    screen_count = []
    
    for i, item in enumerate(df['screen_list']):
        
        name_list = []

        for j, jtem in enumerate((pd.Series(df['screen_list'][i]))):

            if jtem == name:

                name_list.append(name)
    
        screen_count.append(len(name_list))
        
    return(screen_count)

In [ ]:
screens_to_group = ['Credit', 'Loan', 'Profile', 'Saving', 'Verify', 'product_review']
screen_count = []

for s in screens_to_group:
        
    screen_count.append(count_screens(s))

In [ ]:
df['credit_screens'] = screen_count[0]
df['loan_screens'] = screen_count[1]
df['prof_screens'] = screen_count[2]
df['sav_screens'] = screen_count[3]
df['ver_screens'] = screen_count[4]
df['prod_rvw_screens'] = screen_count[5]

In [ ]:
df.head()

In [ ]:
df = df.drop(['user', 'first_open', 'screen_list', 'enrolled_date'], axis=1)


## Exploratory Data Analysis

Starting with the EDA, let's check some of the dataframe parameters:

Now that our dataset is cleaner and easier to work with, let's check the correlation between our numerical variables and our category "enrolled", which is 0 if user didn't subscribe to Premium and 1 if user subscribed to Premium.

We can start by checking some of our dataset parameters, and then define a smaller dataset derived from our main dataset:

In [ ]:
df.describe()

In [ ]:
df_num = df[['age', 'numscreens','enrolled']]

In [ ]:
df_num.head()

The method `pd.DataFrame.corr`is able to calculate the correlation between our variables. Let's plot the outcome in a heatmap:

In [ ]:
sns.heatmap(df_num.corr(method="pearson", numeric_only = False), annot=True);

From the plot, we can see that numscreens and age have low positive and low negative correlation with "enrolled", respectively. 

Let's check the pairplot regarding these variables:

In [ ]:
sns.pairplot(df_num, hue = 'enrolled', height=3);

Now, let's check the correlation between all categorical variables and label "enrolled".

The correlation between two categorical variables is calculated in a different manner compared to the correlation between a numerical and a categorical variable.

In the next steps, we'll use a function that calculates the Chi-square test of independence between two categorical variables. First, we build a cross table between two attributes using the `pd.DataFrame.crosstab` method. Next, this table is used to perform the Chi-square test.

The Chi-Square test returns an array of parameters that are used to understand the correlation between two variables in the following manner:

**Chi-square test**
    Let's consider we have an Hypothesis H0, that states that two variables are not correlated and an Hypothesis H1, which states that both are correlated. We then calculate the p-value regarding both hypotheses. If `p > 0.05`, H1 is true and H0 is rejected. If `p < 0.05`, H1 is discarded and we consider H0 as true

In [ ]:
def findpValues (index, column):
    
    pValues = []
    
    for i in index:

        crosstab = pd.crosstab(index = df[i], columns = df[column])

        chi2 = chi2_contingency(crosstab)
        
        pValues.append(chi2[1])

    return pValues

In [ ]:
p_Values = findpValues(['dayofweek', 'hour', 'minigame', 'used_premium_feature', 'liked', 'dayperiod'], 'enrolled')
p_Values

From the output, we now know that "liked" is the only attribute which is not correlated to "enrolled".








--- 


**Balanced / Imbalanced data**

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(20, 5))

sns.countplot(data=df, x='hour', ax=axes[0])
sns.countplot(data=df, x='dayperiod', ax=axes[1]);

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(20, 5))

sns.countplot(data=df, x='age', ax=axes[0])
sns.countplot(data=df, x='age_group', ax=axes[1]);

---

## ML algorithms

### Data Pipeline

In [ ]:
X = df.drop(['enrolled', 'days_to_enroll'] , axis=1)
y = df['enrolled']

In [ ]:
# X.head()
# y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# categorical columns
cat_cols = ['dayofweek', 'hour', 'age', 'minigame', 'used_premium_feature', 'liked', 'dayperiod', 'age_group']

# numerical columns
num_cols = ['age', 'numscreens']

# numerical columns from transformations
num_cols_screens = ['days_to_enroll', 'credit_screens', 'loan_screens', 'prof_screens', 'sav_screens', 'ver_screens', 'prod_rvw_screens']

In [ ]:
def make_pipeline(cat_cols=[], num_cols=[], classifier = RandomForestClassifier(max_depth=5)):
    
    preprocessor = ColumnTransformer(
        [
            ('ohe', OneHotEncoder(sparse=False, drop='first', handle_unknown='ignore'), cat_cols),
            ('scale', StandardScaler(), num_cols)
        ]
    )

    clf = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', classifier)
        ]
    )

    return clf
#     return preprocessor

<details>
<summary>Click here to understand the contents of our pipeline</summary>
## Contents
</details>

In [ ]:
clf = make_pipeline(cat_cols, num_cols)
clf

In [ ]:
clf.fit(X_train, y_train)

# store results in a dictionary 
results = clf.score(X_test, y_test)

In [ ]:
results

---

---

# Drafts

In [ ]:
df_clean = pd.read_csv('/Users/csergilo/Desktop/strv-ds-academy/app-subscription-strv-academy/datasets/clean_FineTech_appData.csv')

In [ ]:
df_clean.head()

In [ ]:
sorted(df_clean.columns.tolist())

In [ ]:
df_screens = pd.read_csv('/Users/csergilo/Desktop/strv-ds-academy/app-subscription-strv-academy/datasets/top_screens.csv')

In [ ]:
df_screens.head()

---

#### Casting categorical variables

Based on the summary above and on the dtypes and unique values we encountered, let's cast some of our variables as categorical data.  

In [ ]:
df['dayofweek'] = pd.Categorical(df['dayofweek'], categories=list(range(0, 7)))

In [ ]:
# df['hour'] = df['hour'].apply(lambda value: int(value.strip()[0:2]))
df['hour'] = pd.Categorical(df['hour'], categories=list(range(0, 24)))

In [ ]:
# 00:00 -> 05:00 night; 06:00 -> 11:00 morning; 12:00 -> 17:00 afternoon; 18:00 -> 23:00 evening

df['dayperiod'] = df['hour'].apply(lambda value: (('night' if value >= 0 and value <= 5 else 'morning') if value <= 11 else 'afternoon') if value <= 17 else 'evening')
df['dayperiod'] = pd.Categorical(df['dayperiod'], categories=['night', 'morning', 'afternoon', 'evening'])

In [ ]:
df['minigame'] = pd.Categorical(df['minigame'], categories=[0, 1])

In [ ]:
df['used_premium_feature'] = pd.Categorical(df['used_premium_feature'], categories=[0, 1])

In [ ]:
df['liked'] = pd.Categorical(df['liked'], categories=[0, 1])

In [ ]:
df['enrolled'] = pd.Categorical(df['enrolled'], categories=[0, 1])

We can also cast the user ID as a string, so that no erroneous statistical methods are applied to this category.

In [ ]:
# df['user'] = df['user'].apply(str)